# RELANCEZ BIEN LE NOTEBOOK ENTIER A CHAQUE TRAINING POUR GARDER UNE TRACE DES RESULTATS DE CHAQUE MODEL

le model.summary est saved dans saved_models/nom_du_run_wandb

In [45]:
import sys
sys.path.append('../data_cleaning')

from time_series_forecasting import create_dataset
from dataset import build_dataset

In [4]:
import wandb
import tensorflow as tf

In [5]:
from tensorflow.keras.layers import Dense
from keras.layers import Input,Conv1D,MaxPooling1D,Bidirectional,Dropout,CuDNNLSTM,Dense,Input,LSTM, LeakyReLU
from tensorflow.keras import Sequential

### create wandb_api_key.txt and paste your api key inside &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; from https://wandb.ai/settings

In [6]:
file = open('wandb_api_key.txt', 'r')
wandb.login(key=file.readline()) # mettez
file.close()
            

wandb: Currently logged in as: duramann (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\theo-/.netrc


In [38]:
batch_size = 128
epochs = 100
learning_rate = 1e-5
##To edit: 
config = {
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size,
    "window_size": 30,
    "stride":5,
    "number_imu":3,
    "bidirectional":True,
    "lstm_number": 2,
    "lstm_neurons":128,
    "dropout":0.25,
    "dense_number":1,
    "dense_first_neuron":64,#puissance de 2
    "activation_dense":'relu',
    "additional":"usage of convolution"
}

wandb.init(project="AI_Based_Sensor_Information_Fusion", entity="efrai",config=config)

In [31]:
x_train, y_train = build_dataset(0, 50, wandb.config['window_size'],wandb.config['stride'],wandb.config['number_imu'])
x_test, y_test = build_dataset(51,60, wandb.config['window_size'],wandb.config['stride'],wandb.config['number_imu']) 

In [33]:
model = Sequential()
model.add(Input((wandb.config['window_size'], 6*wandb.config['number_imu'])))
model.add(Conv1D(128,11))
model.add(Conv1D(128,11))
model.add(MaxPooling1D(3))
if wandb.config['bidirectional']:
    model.add(Bidirectional(CuDNNLSTM(wandb.config['lstm_neurons'], return_sequences=True)))
else:
    model.add(LSTM(wandb.config['lstm_neurons'], return_sequences=True))
for s in range(wandb.config['lstm_number']-1):
    if wandb.config['bidirectional']:
        model.add(Bidirectional(CuDNNLSTM(wandb.config['lstm_neurons'])))
    else:
        model.add(LSTM(wandb.config['lstm_neurons']))
model.add(Dropout(wandb.config['dropout']))
for d in range(wandb.config['dense_number']):
    model.add(Dense(int(wandb.config['dense_first_neuron'])/(d+1), activation=wandb.config['activation_dense']))
model.add(Dense(6, activation='relu'))
    
    

opt = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=opt, loss='mse')  

In [34]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 20, 128)           25472     
                                                                 
 conv1d_3 (Conv1D)           (None, 10, 128)           180352    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 3, 128)           0         
 )                                                               
                                                                 
 bidirectional_2 (Bidirectio  (None, 3, 256)           264192    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              395264    
 nal)                                                            
                                                      

In [35]:
model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    batch_size = wandb.config['batch_size'],
    epochs=wandb.config['epochs'],
    callbacks=[wandb.keras.WandbCallback(log_batch_frequency=5)])

Epoch 1/100
350/350 [==============================] - 15s 18ms/step - loss: 0.1142 - _timestamp: 1648149769.0000 - _runtime: 41.0000 - val_loss: 0.0556- _timestamp: 16
Epoch 2/100
350/350 [==============================] - 5s 15ms/step - loss: 0.0555 - _timestamp: 1648149775.1549 - _runtime: 47.1549 - val_loss: 0.0474
Epoch 3/100
350/350 [==============================] - 5s 15ms/step - loss: 0.0504 - _timestamp: 1648149780.2817 - _runtime: 52.2817 - val_loss: 0.0441
Epoch 4/100
350/350 [==============================] - 5s 15ms/step - loss: 0.0473 - _timestamp: 1648149785.4225 - _runtime: 57.4225 - val_loss: 0.0414
Epoch 5/100
350/350 [==============================] - 6s 16ms/step - loss: 0.0439 - _timestamp: 1648149790.8169 - _runtime: 62.8169 - val_loss: 0.0377
Epoch 6/100
350/350 [==============================] - 5s 15ms/step - loss: 0.0401 - _timestamp: 1648149796.2817 - _runtime: 68.2817 - val_loss: 0.0331 - _timestamp: 1648149795.7895 -
Epoch 7/100
350/350 [==================

In [46]:
wandb.save('model'+str(wandb.run.name)+'.h5')

AttributeError: 'NoneType' object has no attribute 'name'

In [39]:
wandb.finish()